In [32]:
import sys
sys.path.append('../ames') # path the the directory

import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.api import OLS
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import re
import itertools
%matplotlib inline 

In [33]:
from sklearn.feature_selection import f_regression
from sklearn.linear_model import LinearRegression

In [34]:
import config
from data_prep import clean, add_features

In [35]:
housing_initial = clean(config.HOUSING_CSV)
housing = housing_initial.copy()
housing = add_features(housing)

In [37]:
from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression
ridge = Ridge()
lasso = Lasso()
net   = ElasticNet()

In [38]:
lin_reg_cols = ['Toilets','Wooddecksf','TotRmsAbvGrd','BedroomAbvGr','Showers',
                'FirePlaces','YearBuilt','YearRemodadd',
                'TotalBsmtSF','DecadeBuilt'
               ]

cols_to_dummy = ['Foundation','RoofStyle','SaleCondition',
                 'Neighborhood','Electrical','HouseStyle','SaleType','MSSubClass',
                 'LandContour','PavedDrive','GarageFinish','MSZoning'
                ]

dummy_suffix = ['Qual','Cond','Type']
for col in housing.columns:
    if col not in cols_to_dummy and col not in lin_reg_cols:
        for suff in dummy_suffix:
            #print(re.search(r'%s$'%dummy_suffix, col))
            if re.search(r'%s$'%suff, col) is not None:
                cols_to_dummy.append(col)
                break

In [70]:
ridge_df = housing.copy()
ridge_df = housing[['Neighborhood','OverallQual','BedroomAbvGr','TotalBsmtSF','DecadeBuilt','TotRmsAbvGrd','KitchenQual']]
prices = housing['SalePrice']

dummy_cols = []
lin_cols = []
for col in ridge_df.columns:
    if col in cols_to_dummy:
        dummy_cols.append(col)
    else:
        lin_cols.append(col)

ridge_df = ridge_df.fillna(0.0)
ridge_df = ridge_df.filter(items=lin_cols+dummy_cols)
if len(dummy_cols)>0:
    ridge_df = pd.get_dummies(ridge_df,columns=dummy_cols,drop_first=True)

In [71]:
coefs = []
intercepts = []
R2  = []
alphaRange = [.0005,.0001,.01,1,5]
for alpha in alphaRange:
          ridge.set_params(alpha=alpha)  
          ridge.fit(ridge_df,prices)
          intercepts.append(ridge.intercept_)
          coefs.append(ridge.coef_)
          R2.append(ridge.score(ridge_df,prices)) 

In [72]:
print(R2)

[0.836537310525765, 0.8365373129484573, 0.8365363510320291, 0.8353683629229731, 0.8315730565217787]


In [66]:
dir(ridge)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_check_n_features',
 '_decision_function',
 '_estimator_type',
 '_get_param_names',
 '_get_tags',
 '_more_tags',
 '_preprocess_data',
 '_repr_html_',
 '_repr_html_inner',
 '_repr_mimebundle_',
 '_set_intercept',
 '_validate_data',
 'alpha',
 'coef_',
 'copy_X',
 'fit',
 'fit_intercept',
 'get_params',
 'intercept_',
 'max_iter',
 'n_features_in_',
 'n_iter_',
 'normalize',
 'predict',
 'random_state',
 'score',
 'set_params',
 'solver',
 'tol']